In [605]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

import pickle
import warnings
warnings.filterwarnings('ignore')

In [606]:
df = pd.read_csv(r"C:\Users\techno\OneDrive\Data Trained\Flip Robo (Batch 1836)\Car Price Prediction\Scraped Data\cardekho3.csv")

In [607]:
df

,Unnamed: 0,Brand,Model,Variant,Manufacturing Year,Fuel Type,Driver Kilometers,Number of Owners,Location,Car Price
0,0,2018 Maruti Swift Dzire,ZDI Plus,Manual,2018,Diesel,"55,435 Kms",1st Owner,Ahmedabad,"₹ 7,45,000"
1,1,2019 Maruti Swift Dzire,VXI 1.2 BS IV,Manual,2019,Petrol,"9,386 Kms",1st Owner,Ahmedabad,"₹ 6,50,000"
2,2,2018 Honda City,i-VTEC V,Manual,2018,Petrol,"89,294 Kms",1st Owner,Ahmedabad,"₹ 7,65,000"
3,3,2017Hyundai I20 1.2 Asta,NaN,Manual,2017,Petrol,"24,858 Km",First Owner,Ahmedabad,Rs.7.10 Lakh*
4,4,2018Hyundai Grand I10 1.2 Crdi Sportz,NaN,Manual,2018,Diesel,"47,150 Km",First Owner,Ahmedabad,Rs.5.40 Lakh*
...,...,...,...,...,...,...,...,...,...,...
11848,11848,2015Hyundai Grand i10 AT Asta,NaN,Automatic,2015,Petrol,"20,000 kms",Second Owner,Pune,Rs.4.6 Lakh*
11849,11849,2007Maruti Wagon R VXI BSIII,NaN,Manual,2007,Petrol,"96,585 kms",Third Owner,Pune,Rs.1.1 Lakh*
11850,11850,2017Tata Indica DLS BSIII,NaN,Manual,2017,Diesel,"90,000 kms",Second Owner,Pune,Rs.2 Lakh*
11851,11851,2008Toyota Corolla Altis G,NaN,Manual,2008,Petrol,"90,000 kms",Third Owner,Pune,Rs.2 Lakh*


In [608]:
df.drop(columns='Unnamed: 0', inplace=True)

In [609]:
price_int=[]
loc_int=[]
price_Lakh=[]
loc_Lakh=[]
price_Crore=[]
loc_Crore=[]
price_other=[]
loc_other=[]
price_object=[]
loc_object=[]
for i in range(df.shape[0]):   
    if type(df['Car Price'].loc[i]) == int:
        continue
    elif 'Lakh' in df['Car Price'].loc[i]:
        price_Lakh.append(df['Car Price'].loc[i])
        loc_Lakh.append(i)
    elif 'Cr' in df['Car Price'].loc[i]:
        price_Crore.append(df['Car Price'].loc[i])
        loc_Crore.append(i)
    elif 'Images' in df['Car Price'].loc[i]:
        price_object.append(df['Car Price'].loc[i])
        loc_object.append(i)
    elif 'What' in df['Car Price'].loc[i]:
        price_object.append(df['Car Price'].loc[i])
        loc_object.append(i)
    else:
        price_other.append(df['Car Price'].loc[i])
        loc_other.append(i)

In [610]:
for i in loc_other:
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).replace(',','')
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).replace('Rs.','')
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).replace('₹','')
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).replace('*','')
    df['Car Price'].loc[i] = int((df['Car Price'].loc[i]).replace(' ',''))

In [611]:
for i in loc_Lakh:
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).replace('Rs.','')
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).split('Lakh')[0]
    df['Car Price'].loc[i] = float(df['Car Price'].loc[i])
    df['Car Price'].loc[i] = round((df['Car Price'].loc[i])*100000)

In [612]:
for i in loc_Crore:
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).replace('Rs.','')
    df['Car Price'].loc[i] = (df['Car Price'].loc[i]).split('Cr')[0]
    df['Car Price'].loc[i] = float(df['Car Price'].loc[i])
    df['Car Price'].loc[i] = round((df['Car Price'].loc[i])*10000000)

In [613]:
df.drop(df.index[loc_object],inplace=True)

In [614]:
df = df.reset_index()

In [615]:
df.shape

(11782, 10)

In [616]:
df.dtypes

index                  int64
Brand                 object
Model                 object
Variant               object
Manufacturing Year    object
Fuel Type             object
Driver Kilometers     object
Number of Owners      object
Location              object
Car Price             object
dtype: object

In [617]:
df['Number of Owners'].value_counts()

First Owner               8154
Second Owner              2140
1st Owner                  994
Third Owner                282
2nd Owner                  137
Fourth & Above Owner        48
Unregistered Car Owner      10
Fourth Owner                 9
RTO                          6
3rd Owner                    1
CNG                          1
Name: Number of Owners, dtype: int64

In [618]:
Owners_encode = {
    '1st Owner':1,
    'First Owner':1,
    'Second Owner':2,
    '2nd Owner':2,
    '3rd Owner':3,
    'Third Owner':3,
    'Fourth Owner':4,
    'Fourth & Above Owner':4
}

In [619]:
df['Number of Owners'] = df['Number of Owners'].map(Owners_encode)

In [620]:
df['Number of Owners'].value_counts()

1.0    9148
2.0    2277
3.0     283
4.0      57
Name: Number of Owners, dtype: int64

In [621]:
df.shape

(11782, 10)

In [622]:
df.isnull().sum()

index                     0
Brand                     0
Model                 10644
Variant                   0
Manufacturing Year       31
Fuel Type                 0
Driver Kilometers         0
Number of Owners         17
Location                  0
Car Price                 0
dtype: int64

In [623]:
df['Location'].value_counts()

Mumbai        2838
Delhi NCR     2750
Bangalore     1405
Pune          1069
Chennai        933
Hyderabad      890
Kolkata        691
Ahmedabad      594
Jaipur         456
Chandigarh     156
Name: Location, dtype: int64

In [625]:
for i in range(df.shape[0]):
    try:
        df['Driver Kilometers'].loc[i] = int((df['Driver Kilometers'].loc[i][:-3]).replace(',',''))
    except:
        continue

In [626]:
km_object = []
km_object_loc = []
for i in range(df.shape[0]):
    if type(df['Driver Kilometers'].loc[i]) == str:
        km_object.append(df['Driver Kilometers'].loc[i])
        km_object_loc.append(i)
    elif type(df['Driver Kilometers'].loc[i]) == int:
        continue
    else:
        print(df['Driver Kilometers'].loc[i])
        

In [629]:
df.drop(df.index[km_object_loc],inplace=True)

In [630]:
df = df.reset_index()

In [631]:
for i in range(df.shape[0]):
    if type(df['Driver Kilometers'].loc[i]) == str:
        print('object',df['Driver Kilometers'].loc[i])
    elif type(df['Driver Kilometers'].loc[i]) == int:
        continue
    else:
        print(df['Driver Kilometers'].loc[i])

In [632]:
df.drop(columns=['level_0', 'index'], inplace=True)

In [633]:
df

,Brand,Model,Variant,Manufacturing Year,Fuel Type,Driver Kilometers,Number of Owners,Location,Car Price
0,2018 Maruti Swift Dzire,ZDI Plus,Manual,2018,Diesel,55435,1.0,Ahmedabad,745000
1,2019 Maruti Swift Dzire,VXI 1.2 BS IV,Manual,2019,Petrol,9386,1.0,Ahmedabad,650000
2,2018 Honda City,i-VTEC V,Manual,2018,Petrol,89294,1.0,Ahmedabad,765000
3,2017Hyundai I20 1.2 Asta,NaN,Manual,2017,Petrol,24858,1.0,Ahmedabad,710000
4,2018Hyundai Grand I10 1.2 Crdi Sportz,NaN,Manual,2018,Diesel,47150,1.0,Ahmedabad,540000
...,...,...,...,...,...,...,...,...,...
11770,2015Hyundai Grand i10 AT Asta,NaN,Automatic,2015,Petrol,20000,2.0,Pune,460000
11771,2007Maruti Wagon R VXI BSIII,NaN,Manual,2007,Petrol,96585,3.0,Pune,110000
11772,2017Tata Indica DLS BSIII,NaN,Manual,2017,Diesel,90000,2.0,Pune,200000
11773,2008Toyota Corolla Altis G,NaN,Manual,2008,Petrol,90000,3.0,Pune,200000


In [634]:
year_null = df['Manufacturing Year'].isnull()

In [635]:
year_null_loc=[]
for i in range(df.shape[0]):
    if year_null[i]:
        print('location', i, 'value', df['Manufacturing Year'].loc[i])
        year_null_loc.append(i)

location 1180 value nan
location 1363 value nan
location 1965 value nan
location 5358 value nan
location 5710 value nan
location 5715 value nan
location 5728 value nan
location 5766 value nan
location 5785 value nan
location 5791 value nan
location 5803 value nan
location 5807 value nan
location 5809 value nan
location 5810 value nan
location 5812 value nan
location 7289 value nan
location 7866 value nan
location 10452 value nan
location 10896 value nan
location 10897 value nan
location 11012 value nan
location 11013 value nan
location 11664 value nan
location 11669 value nan
location 11676 value nan
location 11681 value nan
location 11682 value nan
location 11763 value nan
location 11764 value nan
location 11765 value nan
location 11766 value nan


In [639]:
for i in range(df.shape[0]):
    df['Manufacturing Year'].loc[i] = int(df['Brand'].loc[i][0:4])

In [641]:
for i in range(df.shape[0]):
    df['Brand'].loc[i] = df['Brand'].loc[i][4:]

In [642]:
df

,Brand,Model,Variant,Manufacturing Year,Fuel Type,Driver Kilometers,Number of Owners,Location,Car Price
0,Maruti Swift Dzire,ZDI Plus,Manual,2018,Diesel,55435,1.0,Ahmedabad,745000
1,Maruti Swift Dzire,VXI 1.2 BS IV,Manual,2019,Petrol,9386,1.0,Ahmedabad,650000
2,Honda City,i-VTEC V,Manual,2018,Petrol,89294,1.0,Ahmedabad,765000
3,Hyundai I20 1.2 Asta,NaN,Manual,2017,Petrol,24858,1.0,Ahmedabad,710000
4,Hyundai Grand I10 1.2 Crdi Sportz,NaN,Manual,2018,Diesel,47150,1.0,Ahmedabad,540000
...,...,...,...,...,...,...,...,...,...
11770,Hyundai Grand i10 AT Asta,NaN,Automatic,2015,Petrol,20000,2.0,Pune,460000
11771,Maruti Wagon R VXI BSIII,NaN,Manual,2007,Petrol,96585,3.0,Pune,110000
11772,Tata Indica DLS BSIII,NaN,Manual,2017,Diesel,90000,2.0,Pune,200000
11773,Toyota Corolla Altis G,NaN,Manual,2008,Petrol,90000,3.0,Pune,200000


In [643]:
from pathlib import Path  
filepath = Path("C:/Users/techno/OneDrive/Data Trained/Flip Robo (Batch 1836)/Car Price Prediction/Scraped Data/cardekhocleaned.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  